In [1]:
import os
import tifffile as tiff
import numpy as np
from cellpose import models, utils

class ImageProcessor3D:
    def __init__(self, model_type='cyto'):
        """
        Initialize the ImageProcessor3D with a specific Cellpose model type.
        
        :param model_type: str, type of Cellpose model ('cyto', 'nuclei', 'cyto2').
        """
        self.model = models.Cellpose(model_type=model_type)

    def process_image(self, input_path, output_dir):
        """
        Process the input image stack, segment cells, and save results in Cellpose-compatible formats.
        
        :param input_path: str, path to the input TIFF image.
        :param output_dir: str, directory to save the output files.
        """
        # Load the 3D image stack
        image_stack = tiff.imread(input_path)

        # Normalize the image stack to range 0-255
        normalized_stack = self.normalize_image(image_stack)

        # Segment the 3D image stack and retrieve masks and flows
        segmented_stack, flows = self.segment_3d(normalized_stack)

        # Define base name for output files
        base_name = os.path.splitext(os.path.basename(input_path))[0]

        # Save segmentation results for Cellpose GUI
        npy_output_path = os.path.join(output_dir, f"{base_name}_masks.npy")
        output_data = {
            'masks': segmented_stack.astype(np.uint16),  # Segmented masks
            'img': normalized_stack.astype(np.float32),  # Normalized original image
            'flows': flows,                              # Flows for advanced processing
            'filename': os.path.basename(input_path)     # Original image filename
        }
        np.save(npy_output_path, output_data)

        # Save segmented stack in TIFF format
        tiff_output_path = os.path.join(output_dir, f"{base_name}_segmented.tif")
        tiff.imwrite(tiff_output_path, segmented_stack.astype(np.uint16))

        print(f"Processing complete! Outputs saved as:\nNPY: {npy_output_path}\nTIFF: {tiff_output_path}")

    @staticmethod
    def normalize_image(image_stack):
        """
        Normalize image to range 0-255 for Cellpose compatibility.
        
        :param image_stack: 3D numpy array of the input image stack.
        :return: Normalized 3D numpy array.
        """
        img_min = image_stack.min()
        img_max = image_stack.max()
        normalized_stack = (image_stack - img_min) / (img_max - img_min) * 255.0
        return normalized_stack

    def segment_3d(self, image_stack):
        """
        Perform 3D segmentation on the image stack using Cellpose.
        
        :param image_stack: 3D numpy array of the input image stack.
        :return: Tuple of segmented stack and flow fields as a list.
        """
        segmented_stack = np.zeros_like(image_stack, dtype=np.int32)
        all_flows = []

        for z, slice_img in enumerate(image_stack):
            masks, flows, _, _ = self.model.eval(slice_img, diameter=None, channels=[0, 0])
            segmented_stack[z] = masks

            # Append flow fields for each slice
            all_flows.append(flows)  # Keep flows as a list for inhomogeneous shapes

        # Return segmented stack and flow fields (as a list to handle variable shapes)
        return segmented_stack, all_flows


In [2]:
# Usage
if __name__ == "__main__":
    input_image_path = '/Users/nayeb/Downloads/DL_MBL_Data_DP/annotated_images/image_test/TC_19_L2_2-1-1.tif'
    output_directory = '/Users/nayeb/Desktop/Test App'
    
    processor = ImageProcessor3D(model_type='cyto')
    processor.process_image(input_image_path, output_directory)

Processing complete! Outputs saved as:
NPY: /Users/nayeb/Desktop/Test App/TC_19_L2_2-1-1_masks.npy
TIFF: /Users/nayeb/Desktop/Test App/TC_19_L2_2-1-1_segmented.tif
